In [2]:
import os
import pandas as pd
pd.set_option('max_columns', 500)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from functools import reduce
en_stopwords = set(stopwords.words('english')) 
import itertools
import time
import re
import numpy as np
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
my_folder="s3://trident-retention-output/"
folder = 's3://trident-retention-data/askunum/'
if not os.path.exists(os.path.join(os.getcwd(),'outputs')):
    os.makedirs(os.path.join(os.getcwd(),'outputs'))
output_dir=os.path.join(os.getcwd(),'outputs')

In [6]:
x = pd.read_csv(folder + 'askunum_2022_1.csv', nrows=5)
for i in x.loc[x.ParentId=='5003x00002GuQ6tAAF'].sort_values('MessageDate', ascending=False)['TextBody']: 
    print('-'*200)
    print(i)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RE: 851348

Hello Team,

Thank you for your email.

We have completed this termination request for Jonathan Sanchez, with a termination effective date of 06/01/2022. The group will receive a credit of $17.23 on the 07/01/2022 invoice for the premiums paid for their coverage.

Please let us know if there is anything else that we can assist you with.

Thank you,

The Ask Unum Team

Client Success Organization
1-800-Ask-Unum (1-800-275-8686)
AskUnum@unum.com

Unum COVID-19 Response - Click here
How to File a Claim Online - Click here

We appreciate the opportunity to meet your benefit needs. If you have any questions, we have experienced Service Specialists available to help you Monday through Friday, 8 a.m. to 8 p.m. Eastern Time.



--------------- Original Message ---------------
From: FL

## id counts and issue durations

#### load the data, process into issue durations and counts
- data format for each year is a bit different. Thus, we standardize and then apply the function to obtain counts and durations. 

In [4]:
def load_askunum_df(folder, year, usecols=None, nrows=None): 
    if year == 2018: # ['ID', 'PARENTID', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE']
        askunum_df = pd.read_csv(folder + 'askunum_2018.csv', encoding='latin-1', usecols=usecols, nrows=nrows)
       
    if year == 2019: 
        askunum_df = pd.concat([pd.read_csv(folder + 'askunum_2019_{}.csv'.format(i), encoding='latin-1', usecols=usecols, nrows=nrows) for i in range(1, 4)]) 
        
    if year == 2020:  
        askunum_df = pd.concat([pd.read_csv(folder + 'unnested_2020_{}_customer.csv'.format(i), encoding='latin-1', usecols=usecols, nrows=nrows) for i in range(10)])

    if year == 2021: 
        askunum_df = pd.concat([pd.read_csv(folder + 'unnested_2021_{}_customer.csv'.format(i), encoding='latin-1', usecols=usecols, nrows=nrows) for i in range(10)]) 
        
    if year == 2022: 
        askunum_df = pd.concat([pd.read_csv(folder + 'askunum_2022_{}.csv'.format(i), encoding='latin-1', usecols=usecols, nrows=nrows) for i in range(0, 4)])
        
    return askunum_df

def pipeline_askunum_counts_and_duration(folder, year, usecols=False, parent_id=False, target_columns=False): 
    if target_columns == False: 
        target_columns = ['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'account_id']
    
    def helper_get_counts_and_duration(askunum_df):
        """_summary_

        Args:
            askunum_df (pd.DataFrame): dataframe with ['Id', 'ParentId', 'account_id', 'CreatedDate', 'ClosedDate'] as columns

        Returns:
            pd.DataFrame with ['account_id', 'year', 'month', 'id_count', 'parent_id_count', 'askunum_days']
        """
        # issue counts by created date
        askunum_df['CreatedDate'] = pd.to_datetime(askunum_df['CreatedDate'])
        askunum_df['year'] = askunum_df.CreatedDate.apply(lambda x: x.year)
        askunum_df['month'] = askunum_df.CreatedDate.apply(lambda x: x.month)
        
        email_counts_by_month = askunum_df.groupby(['account_id', 'year', 'month'])[['Id']].count()
        issue_counts_by_month = askunum_df.drop('Id', axis=1).drop_duplicates().groupby(['account_id', 'year', 'month'])[['ParentId']].count()
        combined_df = email_counts_by_month.join(issue_counts_by_month)
        combined_df.rename({"Id":'askunum_id_count', 'ParentId':'askunum_parentid_count'}, axis=1, inplace=True)
        email_counts_by_month, issue_counts_by_month = None, None
        
        # completed issue durations
        askunum_df = askunum_df.loc[~askunum_df.ClosedDate.isna()]
        askunum_df['ClosedDate'] = pd.to_datetime(askunum_df['ClosedDate'])
        askunum_df['askunum_days'] = (askunum_df['ClosedDate'] - askunum_df['CreatedDate']).apply(lambda x: (x.days * 24 + x.seconds / 3600)/24)
        issue_days_by_month = askunum_df.groupby(['account_id', 'year', 'month'])[['askunum_days']].sum()
        combined_df = combined_df.join(issue_days_by_month, how='outer')
        combined_df[['askunum_id_count', 'askunum_parentid_count', 'askunum_days']].fillna(0, inplace=True)
        
        return combined_df 
    
    if year in [2018, 2019]: 
        if usecols==False: 
            usecols = ['ID', 'PARENTID', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE']
        if parent_id==False: 
            parent_id = 'PARENTID'
     
    if year in [2020, 2021]: 
        if usecols==False: 
            usecols = ['Id', 'ParentId', 'CreatedDate', 'ClosedDate']
        if parent_id==False: 
            parent_id = 'ParentId'
            
    if year in [2022]: 
        if usecols==False: 
            usecols = ['Id', 'ParentId', 'Parent.CreatedDate', 'Parent.ClosedDate']
        if parent_id==False: 
            parent_id = 'ParentId'
            
    askunum_df = load_askunum_df(folder, year, usecols=usecols).rename({parent_id: 'ParentId'}, axis=1) #use ParentId as the standard
    account_mapping = pd.read_csv(folder + '{}ParentAccount.csv'.format(year), usecols=['ParentId', 'Parent.AccountId']).drop_duplicates().dropna()
    askunum_df = pd.merge(askunum_df, account_mapping, on='ParentId')
    cols = ['ParentId' if i == parent_id else i for i in usecols] + ['Parent.AccountId']
    print(target_columns, cols)
    askunum_df = askunum_df.rename(dict(zip(cols, target_columns)), axis=1)
    print(askunum_df.shape)
     
    askunum_features = helper_get_counts_and_duration(askunum_df)
    askunum_features.to_csv(os.path.join(output_dir , 'askunum_issue_count_and_duration_{}.csv'.format(year)))
    print(askunum_features.shape)
    return askunum_df, askunum_features

In [5]:
# if False: 
x = pipeline_askunum_counts_and_duration(folder, 2018)
x = pipeline_askunum_counts_and_duration(folder, 2019)
x = pipeline_askunum_counts_and_duration(folder, 2020)
x = pipeline_askunum_counts_and_duration(folder, 2021)
x = pipeline_askunum_counts_and_duration(folder, 2022)

['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'account_id'] ['ID', 'ParentId', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE', 'Parent.AccountId']
(1062502, 5)


/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(203116, 3)
['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'account_id'] ['ID', 'ParentId', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE', 'Parent.AccountId']
(2680801, 5)
(208774, 3)
['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'account_id'] ['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'Parent.AccountId']
(1220677, 5)
(208675, 3)
['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'account_id'] ['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'Parent.AccountId']
(1189271, 5)
(205766, 3)
['Id', 'ParentId', 'CreatedDate', 'ClosedDate', 'account_id'] ['Id', 'ParentId', 'Parent.CreatedDate', 'Parent.ClosedDate', 'Parent.AccountId']
(556079, 5)
(97755, 3)


In [6]:
# take a look at the df and the features
x[0].head()

,Id,ParentId,CreatedDate,ClosedDate,account_id,year,month
0,02s3x00001pNsfoAAC,5003x00002GCgn4AAD,2022-04-07 02:43:01+00:00,2022-04-07T14:21:34.000Z,0010c00001tugMbAAI,2022,4
1,02s3x00001pNoBSAA0,5003x00002GCgn4AAD,2022-04-07 02:43:01+00:00,2022-04-07T14:21:34.000Z,0010c00001tugMbAAI,2022,4
2,02s3x00001pNoBXAA0,5003x00002GCgn9AAD,2022-04-07 02:44:34+00:00,2022-04-12T20:02:04.000Z,0010c00002CoRGoAAN,2022,4
3,02s3x00001pwSJGAA2,5003x00002GCgn9AAD,2022-04-07 02:44:34+00:00,2022-04-12T20:02:04.000Z,0010c00002CoRGoAAN,2022,4
4,02s3x00001pNsR9AAK,5003x00002GCgoHAAT,2022-04-07 03:04:47+00:00,2022-04-07T14:19:00.000Z,00130000018kIK1AAM,2022,4


In [7]:
x[1].head()

askunum_id_count  askunum_parentid_count  \
account_id         year month                                             
0010c00001sr7B9AAI 2022 2                     6                       2   
0010c00001sr7BAAAY 2022 5                     2                       1   
0010c00001sr7BBAAY 2022 1                     2                       1   
0010c00001sr7BGAAY 2022 1                     2                       1   
                        5                    15                       4   

                               askunum_days  
account_id         year month                
0010c00001sr7B9AAI 2022 2          0.240347  
0010c00001sr7BAAAY 2022 5          2.190463  
0010c00001sr7BBAAY 2022 1          6.095949  
0010c00001sr7BGAAY 2022 1         11.419028  
                        5          1.724549

In [8]:
x[1].shape

(97755, 3)

In [9]:
# combined the askunum counts and duration data
combined_monthly_counts = []
for i in [2018, 2019, 2020, 2021, 2022]: 
    df = pd.read_csv(os.path.join(output_dir , 'askunum_issue_count_and_duration_{}.csv'.format(i)))
    print(i, df.shape)
    combined_monthly_counts.append(df)
combined_monthly_counts = pd.concat(combined_monthly_counts)
combined_monthly_counts.to_csv(os.path.join(output_dir ,'askunum_issue_count_and_duration_combined.csv'), index=False)

2018 (203116, 6)
2019 (208774, 6)
2020 (208675, 6)
2021 (205766, 6)
2022 (97755, 6)


In [10]:
def extract_subtype_count_duration(year): 
    target_columns = ['ParentId', 'CreatedDate', 'ClosedDate', 'SUB_TYPE_TXT__c']
            
    if year in [2018, 2019]: 
        usecols = ['PARENTID', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE', 'PARENT.SUB_TYPE_TXT__C']
        
    if year in [2020, 2021]: 
        usecols = ['ParentId', 'CreatedDate', 'ClosedDate', 'SUB_TYPE_TXT__c']
        
    if year in [2022]: 
        usecols = ['ParentId', 'Parent.CreatedDate', 'Parent.ClosedDate', 'Parent.SUB_TYPE_TXT__c']
        
    askunum_df = load_askunum_df(folder, year, usecols=usecols)
    askunum_df.rename(dict(zip(usecols, target_columns)), axis=1, inplace=True)
    account_mapping = pd.read_csv(folder + '{}ParentAccount.csv'.format(year), usecols=['ParentId', 'Parent.AccountId']).drop_duplicates().dropna()
    askunum_df = pd.merge(askunum_df, account_mapping, on='ParentId')
    askunum_df.rename({'Parent.AccountId' : 'account_id'}, axis=1, inplace=True)

    # per parent_id created dates durations, and subtypes counts
    askunum_df = askunum_df.groupby(['account_id', 'ParentId']).agg({'CreatedDate': 'first', 'ClosedDate':'first', 'SUB_TYPE_TXT__c' : 'first'})
    askunum_df['CreatedDate'] = pd.to_datetime(askunum_df['CreatedDate'])
    askunum_df['ClosedDate'] = pd.to_datetime(askunum_df['ClosedDate'])
    askunum_df['count'] = 1
    askunum_df['askunum_days'] = (askunum_df['ClosedDate'] - askunum_df['CreatedDate']).apply(lambda x: (x.days * 24 + x.seconds / 3600)/24)

    # per account, per subtype txt_c sum
    askunum_count_pivot = pd.pivot_table(askunum_df, index='account_id', columns='SUB_TYPE_TXT__c', values='count', aggfunc='sum').fillna(0)
    askunum_days_pivot = pd.pivot_table(askunum_df, index='account_id', columns='SUB_TYPE_TXT__c', values='askunum_days', aggfunc='sum').fillna(0)
    cols = list(askunum_count_pivot.columns)
    new_count_cols = ["SUB_TYPE_TXT__c_{}".format(i) for i in cols]
    new_days_cols = ["SUB_TYPE_TXT__c_{}_duration".format(i) for i in cols]
    askunum_count_pivot = askunum_count_pivot.rename(dict(zip(cols, new_count_cols)), axis=1) 
    askunum_days_pivot = askunum_days_pivot.rename(dict(zip(cols, new_days_cols)), axis=1) 

    assert askunum_count_pivot.shape[0] == len(askunum_count_pivot.index.unique())
    assert askunum_days_pivot.shape[0] == len(askunum_days_pivot.index.unique())

    # join and save
    askunum_subtypes_df = askunum_count_pivot.join(askunum_days_pivot)
    askunum_subtypes_df.to_csv(os.path.join(output_dir ,'account_level_askunum_subtypes_{}.csv'.format(year)))
    return askunum_subtypes_df

In [11]:
askunum_subtypes_df_2018 = extract_subtype_count_duration(2018)
askunum_subtypes_df_2019 = extract_subtype_count_duration(2019)
askunum_subtypes_df_2020 = extract_subtype_count_duration(2020)
askunum_subtypes_df_2021 = extract_subtype_count_duration(2021)
askunum_subtypes_df_2022 = extract_subtype_count_duration(2022)

#### monthly subtypes

In [12]:
# get all possible subtypes and create a mapping file consistent with indicators -> map sub types to indicators/ groups instead of individual
sub_type_columns = []
for i in range(2018, 2023):
    subtype_temp = pd.read_csv(os.path.join(output_dir,'account_level_askunum_subtypes_{}.csv'.format(i)), nrows=1)
    sub_type_columns += list(subtype_temp.columns)

# sub_type_columns = list(set(sub_type_columns))
sub_type_columns = [c for c in sub_type_columns if '_duration' not in c]
sub_type_columns = [re.sub(r'[^A-Za-z0-9_]+', '', c) for c in sub_type_columns]

In [13]:
askunum_subtype_to_indicator_mapping = pd.read_csv(folder + 'askunum_subtype_to_indicators.csv')
askunum_subtype_to_indicator_mapping

,feature,indicator
0,SUB_TYPE_TXT__c_LateNoticeorCollections,[Indicator] AskUnum Bill Payment Issue
1,SUB_TYPE_TXT__c_MissingorSkippedPayment,[Indicator] AskUnum Bill Payment Issue
2,SUB_TYPE_TXT__c_BillNotReceived,[Indicator] AskUnum Billing Issues
3,SUB_TYPE_TXT__c_EmployeeCodingDiscrepancy,[Indicator] AskUnum Billing Issues
4,SUB_TYPE_TXT__c_PolicyLevelDiscrepancy,[Indicator] AskUnum Billing Issues
...,...,...
136,SUB_TYPE_TXT__c_RequestInfo,askunum_other
137,SUB_TYPE_TXT__c_StarmountRewriteInquiry,askunum_starmount
138,SUB_TYPE_TXT__c_TechnologySupport,askunum_website
139,SUB_TYPE_TXT__c_TerminationLetterUnumAppointment,askunum_broker_change


In [14]:
# load in the mapping
askunum_subtype_to_indicator_mapping = pd.read_csv(folder + 'askunum_subtype_to_indicators.csv')
askunum_subtype_to_indicator_mapping = dict(zip(list(askunum_subtype_to_indicator_mapping.feature.apply(lambda x: x.replace('SUB_TYPE_TXT__c_', ''))), \
                                                list(askunum_subtype_to_indicator_mapping.indicator.apply(lambda x: x.replace('[Indicator]', '')).apply(lambda x: x.strip()))))
askunum_subtype_to_indicator_mapping

{'LateNoticeorCollections': 'AskUnum Bill Payment Issue',
 'MissingorSkippedPayment': 'AskUnum Bill Payment Issue',
 'BillNotReceived': 'AskUnum Billing Issues',
 'EmployeeCodingDiscrepancy': 'AskUnum Billing Issues',
 'PolicyLevelDiscrepancy': 'AskUnum Billing Issues',
 'PremiumDiscrepancy': 'AskUnum Billing Issues',
 'Refund': 'AskUnum Billing Issues',
 'WriteOff': 'AskUnum Billing Issues',
 'BrokerofRecordChangeBOR': 'Broker of Record Change',
 'RetroBORChange': 'Broker of Record Change',
 'ServicingBrokerofRecordChangeSBOR': 'Broker of Record Change',
 'BORPacket': 'Broker Requests Packet',
 'AdditionalClaimInformationReceived': 'Claim Inquiries',
 'BenefitPaymentStatus': 'Claim Inquiries',
 'ClaimStatus': 'Claim Inquiries',
 'ElectronicEOIInquiry': 'EOI Requests',
 'EOIStatus': 'EOI Requests',
 'EOISubmission': 'EOI Requests',
 'EvidenceofInsurability': 'EOI Requests',
 'NewPlanAdministrator': 'New Plan Admin',
 'ACHInquiryConfirmation': 'askunum_billing',
 'BillHold': 'askunum_bi

In [15]:
def extract_askunum_subtype_group_features(year): 
    """get count and duration from askunum subtpyes by month

    Args:
        year (_type_): _description_

    Returns:
        _type_: multi-column index -> (issue_count, [indicator] xyz)
    """
    print('starting askunum subtype group extract', year)
    target_columns = ['ParentId', 'CreatedDate', 'ClosedDate', 'SUB_TYPE_TXT__c']
                
    if year in [2018, 2019]: 
        usecols = ['PARENTID', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE', 'PARENT.SUB_TYPE_TXT__C']
        
    if year in [2020, 2021]: 
        usecols = ['ParentId', 'CreatedDate', 'ClosedDate', 'SUB_TYPE_TXT__c']
        
    if year in [2022]: 
        usecols = ['ParentId', 'Parent.CreatedDate', 'Parent.ClosedDate', 'Parent.SUB_TYPE_TXT__c']

    # standardize data
    askunum_df = load_askunum_df(folder, year, usecols=usecols)
    askunum_df.rename(dict(zip(usecols, target_columns)), axis=1, inplace=True)
    account_mapping = pd.read_csv(folder + '{}ParentAccount.csv'.format(year), usecols=['ParentId', 'Parent.AccountId']).drop_duplicates().dropna()
    askunum_df = pd.merge(askunum_df, account_mapping, on='ParentId')
    askunum_df.rename({'Parent.AccountId' : 'account_id'}, axis=1, inplace=True)

    # askunum_df.head()

    # group by account and parentid
    askunum_df['askunum_email_count'] = 1
    askunum_df = askunum_df.groupby(['account_id', 'ParentId']).agg({'CreatedDate': 'first', 'ClosedDate':'first', 'SUB_TYPE_TXT__c' : 'first', 'askunum_email_count' : 'sum'})
    askunum_df['CreatedDate'] = pd.to_datetime(askunum_df['CreatedDate'])
    askunum_df['ClosedDate'] = pd.to_datetime(askunum_df['ClosedDate'])
    askunum_df['askunum_issue_count'] = 1
    askunum_df['askunum_issue_duration'] = (askunum_df['ClosedDate'] - askunum_df['CreatedDate']).apply(lambda x: (x.days * 24 + x.seconds / 3600)/24)

    # askunum_df.head()

    # clean up and transfomr the sub_types -> TODO some mapping might be incorrect
    askunum_df['askunum_subtype_group'] = askunum_df['SUB_TYPE_TXT__c'].str.replace('[^A-Za-z0-9_]+', '')
    askunum_df['askunum_subtype_group'] = askunum_df['askunum_subtype_group'].map(askunum_subtype_to_indicator_mapping)
    askunum_df['askunum_subtype_group'] = askunum_df['askunum_subtype_group'].str.replace('[^A-Za-z0-9_[]]+', '')

    askunum_df['askunum_subtype_group'].value_counts()

    # set year month the begining of the month
    askunum_df['year_month'] = pd.to_datetime(askunum_df['CreatedDate'].apply(lambda x: str(x.year) + '-' + str(x.month) ))

    # for each account, year-month, and subtype group get the features you want -> then pviot by askunum_subtype_group to create a time based feature for each account
    askunum_group = askunum_df.groupby(['account_id', 'year_month', 'askunum_subtype_group'])['askunum_issue_count', 'askunum_issue_duration', 'askunum_email_count'].sum()
    askunum_pivot = askunum_group.reset_index().pivot(index=['account_id', 'year_month'], columns=['askunum_subtype_group'])
    askunum_pivot.to_csv(os.path.join(output_dir , 'askunum_subtype_groups_by_account_year_month_{}.csv'.format(year)))
    return askunum_pivot

In [16]:
askunum_subtype_groups_2018 = extract_askunum_subtype_group_features(2018)
askunum_subtype_groups_2019 = extract_askunum_subtype_group_features(2019)
askunum_subtype_groups_2020 = extract_askunum_subtype_group_features(2020)
askunum_subtype_groups_2021 = extract_askunum_subtype_group_features(2021)
askunum_subtype_groups_2022 = extract_askunum_subtype_group_features(2022)

starting askunum subtype group extract 2018


/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


starting askunum subtype group extract 2019
starting askunum subtype group extract 2020
starting askunum subtype group extract 2021
starting askunum subtype group extract 2022


In [17]:
# combine
askunum_subtype_groups_combined = pd.concat([askunum_subtype_groups_2018, askunum_subtype_groups_2019, askunum_subtype_groups_2020, askunum_subtype_groups_2021, askunum_subtype_groups_2022])
askunum_subtype_groups_combined.columns = [c[0] + '_' + c[1] for c in askunum_subtype_groups_combined.columns]
askunum_subtype_groups_combined = askunum_subtype_groups_combined.reset_index()

In [18]:
unique_account_ids = askunum_subtype_groups_combined.account_id.unique()
year_month_range = pd.date_range('2018-01', '2022-06', freq='MS')
account_year_month_index = pd.DataFrame(list(itertools.product(unique_account_ids, year_month_range)))
account_year_month_index = account_year_month_index.set_index([0, 1])
account_year_month_index.index.rename(['account_id', 'year_month'], inplace=True)

In [19]:
# merge the subtypes on the index
askunum_subtype_groups_combined = askunum_subtype_groups_combined.set_index(['account_id', 'year_month'])
askunum_subtype_groups_extended = pd.merge(askunum_subtype_groups_combined, account_year_month_index, how='right', left_index=True, right_index=True)

In [20]:
# check that the sum of each column remains the same
orig_sum = askunum_subtype_groups_combined.sum()
new_sum = askunum_subtype_groups_extended.sum()
print((orig_sum - new_sum).min(), (orig_sum - new_sum).max())

-3.4924596548080444e-10 9.313225746154785e-10


In [1]:
unique_account_ids.shape

NameError: name 'unique_account_ids' is not defined

In [22]:
askunum_subtype_groups_combined.columns

Index(['askunum_email_count_AskUnum Bill Payment Issue',
       'askunum_email_count_AskUnum Billing Issues',
       'askunum_email_count_Broker Requests Packet',
       'askunum_email_count_Broker of Record Change',
       'askunum_email_count_Claim Inquiries',
       'askunum_email_count_EOI Requests',
       'askunum_email_count_New Plan Admin',
       'askunum_email_count_askunum_billing',
       'askunum_email_count_askunum_broker_change',
       'askunum_email_count_askunum_broker_commission',
       'askunum_email_count_askunum_broker_other',
       'askunum_email_count_askunum_claims',
       'askunum_email_count_askunum_employee_maintenance',
       'askunum_email_count_askunum_forms',
       'askunum_email_count_askunum_other',
       'askunum_email_count_askunum_plan_change',
       'askunum_email_count_askunum_plan_inquiry',
       'askunum_email_count_askunum_starmount',
       'askunum_email_count_askunum_termination',
       'askunum_email_count_askunum_website',
       

In [23]:
year=2022
"""get count and duration from askunum subtpyes by month

Args:
    year (_type_): _description_

Returns:
    _type_: multi-column index -> (issue_count, [indicator] xyz)
"""
print('starting askunum subtype group extract', year)
target_columns = ['ParentId', 'CreatedDate', 'ClosedDate', 'SUB_TYPE_TXT__c']

if year in [2018, 2019]: 
    usecols = ['PARENTID', 'PARENT.CREATEDDATE', 'PARENT.CLOSEDDATE', 'PARENT.SUB_TYPE_TXT__C']

if year in [2020, 2021]: 
    usecols = ['ParentId', 'CreatedDate', 'ClosedDate', 'SUB_TYPE_TXT__c']

if year in [2022]: 
    usecols = ['ParentId', 'Parent.CreatedDate', 'Parent.ClosedDate', 'Parent.SUB_TYPE_TXT__c']

# standardize data
askunum_df = load_askunum_df(folder, year, usecols=usecols)
askunum_df.rename(dict(zip(usecols, target_columns)), axis=1, inplace=True)
account_mapping = pd.read_csv(folder + '{}ParentAccount.csv'.format(year), usecols=['ParentId', 'Parent.AccountId']).drop_duplicates().dropna()
askunum_df = pd.merge(askunum_df, account_mapping, on='ParentId')
askunum_df.rename({'Parent.AccountId' : 'account_id'}, axis=1, inplace=True)

askunum_df.head(2)

starting askunum subtype group extract 2022


,ParentId,CreatedDate,ClosedDate,SUB_TYPE_TXT__c,account_id
0,5003x00002GCgn4AAD,2022-04-07T02:43:01.000Z,2022-04-07T14:21:34.000Z,ACH Inquiry/Confirmation,0010c00001tugMbAAI
1,5003x00002GCgn4AAD,2022-04-07T02:43:01.000Z,2022-04-07T14:21:34.000Z,ACH Inquiry/Confirmation,0010c00001tugMbAAI


In [24]:
askunum_df['askunum_email_count'] = 1
askunum_df = askunum_df.groupby(['account_id', 'ParentId']).agg({'CreatedDate': 'first', 'ClosedDate':'first', 'SUB_TYPE_TXT__c' : 'first', 'askunum_email_count' : 'sum'})
askunum_df['CreatedDate'] = pd.to_datetime(askunum_df['CreatedDate'])
askunum_df['ClosedDate'] = pd.to_datetime(askunum_df['ClosedDate'])

askunum_df.head(2)

CreatedDate  \
account_id         ParentId                                       
0010c00001sr7B9AAI 5003x00002Efu6sAAB 2022-02-07 14:05:19+00:00   
                   5003x00002EfukKAAR 2022-02-07 14:59:58+00:00   

                                                     ClosedDate  \
account_id         ParentId                                       
0010c00001sr7B9AAI 5003x00002Efu6sAAB 2022-02-07 14:17:29+00:00   
                   5003x00002EfukKAAR 2022-02-07 17:28:41+00:00   

                                      SUB_TYPE_TXT__c  askunum_email_count  
account_id         ParentId                                                 
0010c00001sr7B9AAI 5003x00002Efu6sAAB  Filing Inquiry                    4  
                   5003x00002EfukKAAR  Filing Inquiry                    2

In [25]:
# askunum_df['askunum_issue_count'] = 1
# askunum_df['askunum_issue_duration'] = (askunum_df['ClosedDate'] - askunum_df['CreatedDate']).apply(lambda x: (x.days * 24 + x.seconds / 3600)/24)

# clean up and transfomr the sub_types -> TODO some mapping might be incorrect
# askunum_df['askunum_subtype_group'] = askunum_df['SUB_TYPE_TXT__c'].str.replace('[^A-Za-z0-9_]+', '')
# askunum_df['askunum_subtype_group'] = askunum_df['askunum_subtype_group'].map(askunum_subtype_to_indicator_mapping)
# askunum_df['askunum_subtype_group'] = askunum_df['askunum_subtype_group'].str.replace('[^A-Za-z0-9_[]]+', '')
# set year month the begining of the month
askunum_df['year_month'] = pd.to_datetime(askunum_df['CreatedDate'].apply(lambda x: str(x.year) + '-' + str(x.month) ))
askunum_df.head(2)

CreatedDate  \
account_id         ParentId                                       
0010c00001sr7B9AAI 5003x00002Efu6sAAB 2022-02-07 14:05:19+00:00   
                   5003x00002EfukKAAR 2022-02-07 14:59:58+00:00   

                                                     ClosedDate  \
account_id         ParentId                                       
0010c00001sr7B9AAI 5003x00002Efu6sAAB 2022-02-07 14:17:29+00:00   
                   5003x00002EfukKAAR 2022-02-07 17:28:41+00:00   

                                      SUB_TYPE_TXT__c  askunum_email_count  \
account_id         ParentId                                                  
0010c00001sr7B9AAI 5003x00002Efu6sAAB  Filing Inquiry                    4   
                   5003x00002EfukKAAR  Filing Inquiry                    2   

                                      year_month  
account_id         ParentId                       
0010c00001sr7B9AAI 5003x00002Efu6sAAB 2022-02-01  
                   5003x00002EfukKAAR 2022-02-01

#### preprocessing by year
- bag of words 
- stem

In [ ]:
def askunum_stemmed_bag_of_words(folder, year, nrows=None): 
    if year in [2018, 2019]: 
        id = 'ID'
        parent_id = 'PARENTID'
        text_body = 'TEXTBODY'
        created_date = 'PARENT.CREATEDDATE'      

    if year in [2020, 2021]: 
        id = 'Id'
        parent_id = 'ParentId'
        text_body = 'TextBody'
        created_date = 'CreatedDate' 

    if year in [2022]:
        id = 'Id'
        parent_id = 'ParentId'
        text_body = 'TextBody'
        created_date = 'Parent.CreatedDate'  

    askunum_text = load_askunum_df(folder, year, usecols = [id, parent_id, text_body, created_date], nrows=nrows)
    askunum_text = askunum_text.rename(dict(zip([id, parent_id, text_body, created_date], ['Id', 'ParentId', 'TextBody', 'CreatedDate'])), axis=1)

    askunum_text['TextBody'] = askunum_text['TextBody'].fillna("").astype(str).str.lower()
    askunum_text['TextBody'] = askunum_text['TextBody'].apply(lambda x: x.split('from:')[0])  # split by from:

    askunum_text['CreatedDate'] = pd.to_datetime(askunum_text['CreatedDate'])
    askunum_text['year'] = askunum_text.CreatedDate.apply(lambda x: x.year)
    askunum_text['month'] = askunum_text.CreatedDate.apply(lambda x: x.month)

    # remove phrases
    phrases = [
              'caution external email: this email originated from outside of the organization. do not click links or open attachments unless you recognize the sender and know the content is safe.', 
              'this email message and its attachments are for the sole use of the intended recipient or recipients and may contain confidential information. if you have received this email in error, please notify the sender and delete this message.',
        ]

    for p in phrases:
        askunum_text['TextBody']= askunum_text['TextBody'].str.replace(p, ' ', regex=False)

    # replace special
    for s in ['\n', '\t', '\r', '\b', '\f']:
        askunum_text['TextBody'] = askunum_text['TextBody'].str.replace(s, ' ', regex=False)

    askunum_text['TextBody'] = askunum_text['TextBody'].str.replace('[^A-Za-z]', ' ', regex=True) # replace non-alphanumeric with space
    askunum_text['TextBody'] = askunum_text['TextBody'].str.replace(r'[ ]+', ' ', regex=True) # replace more than one space with a single space
    askunum_text['TextBodyBag'] = askunum_text['TextBody'].apply(lambda x: set(x.split(' ')) - en_stopwords) #remove stopwords
    askunum_text['TextBodyBag'] = askunum_text['TextBodyBag'].apply(lambda x: set([i for i in x if len(i)>2])) # remove short words

    my_union = lambda x: reduce(set.union, x)
    askunum_text = askunum_text.groupby(['ParentId', 'year', 'month'])['TextBodyBag'].agg(my_union) #aggregate by parent_id, year, and month

    # stem
    ps = PorterStemmer() 
    askunum_text = askunum_text.apply(lambda x: set([ps.stem(w) for w in x]))
    askunum_text.to_csv(os.path.join(output_dir ,'askunum_textbody_stemmed_bag_of_words_{}.csv'.format(year)))

    print(askunum_text.shape)
    return askunum_text


run_stemming = True
if run_stemming: 
      for year in range(2018, 2023): 
            askunum_stemmed_bag_of_words(folder, year)
            print(year, 'complete')